In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
schema=pd.read_csv("/kaggle/input/so-survey-2017/survey_results_schema.csv")
schema.head()

In [ ]:
pd.options.display.max_colwidth = 100
schema[schema['Column']=="YearsCodedJobPast"]["Question"]

In [ ]:
data=pd.read_csv("/kaggle/input/so-survey-2017/survey_results_public.csv")
data.head()

In [ ]:
columns = data.columns
print(columns[1:100])
print(columns[100:200])


In [ ]:
edu_type = data['EducationTypes'].value_counts().reset_index()
edu_type.head()

In [ ]:
#First of all let's clean the data
#Here we are no dealing with any null value as it will automatically not be considered

def clean_series(df,col_name):
    """
    inputs: 
    df: dataframe which you want to manipulate
    col_name: name of the column which you want to clean the data
    
    outputs:
    df2: Panda's dataframe with the unique element seperated and
         their count
    """
    temp = df[col_name]
    temp = temp.dropna().reset_index()
    temp = temp[col_name].str.split(';')
    emp_list=[]
    for i in range(len(temp)):
        emp_list += temp[i]

    for i in range(len(emp_list)):
        emp_list[i] = emp_list[i].strip()

    emp_set = list(set(emp_list))    
    emp_set

    c = Counter(emp_list)
    print(c.items())
    df2 = pd.DataFrame({'Methods':list(c.keys()),'Counts':list(c.values())})
    
    return df2

In [ ]:
from collections import Counter
methods = clean_series(data,'EducationTypes').sort_values(by='Counts')
methods


## Gender Job Satisfaction

In [ ]:
data["Gender"].value_counts()[0:10].plot(kind='bar',figsize=(12,10));

In [ ]:
# Select only the 2 gender with the most participants
df = data[(data["Gender"]=="Male") | (data["Gender"]=="Female")]

# Plot percentage of CareerSatisfaction per Gender
grouped = df.groupby(['Gender'], sort=False)
JobSatisfaction_counts = grouped['CareerSatisfaction'].value_counts(normalize=True, sort=False)

Satisfaction_data = [
    {'CareerSatisfaction': JobSatisfaction, 'Gender': Gender, 'percentage': percentage*100} for 
    (Gender, JobSatisfaction), percentage in dict(JobSatisfaction_counts).items()
]

df_Satisfaction = pd.DataFrame(Satisfaction_data)

p = sns.barplot(x="CareerSatisfaction", y="percentage", hue="Gender", data=df_Satisfaction)
_ = plt.setp(p.get_xticklabels(), rotation=90)  # Rotate labels

In [ ]:
# Plot percentage of JobSatisfaction per Gender
grouped = df.groupby(['Gender'], sort=False)
JobSatisfaction_counts = grouped['JobSatisfaction'].value_counts(normalize=True, sort=False)

Satisfaction_data = [
    {'JobSatisfaction': JobSatisfaction, 'Gender': Gender, 'percentage': percentage*100} for 
    (Gender, JobSatisfaction), percentage in dict(JobSatisfaction_counts).items()
]

df_Satisfaction = pd.DataFrame(Satisfaction_data)

p = sns.barplot(x="JobSatisfaction", y="percentage", hue="Gender", data=df_Satisfaction)
_ = plt.setp(p.get_xticklabels(), rotation=90)  # Rotate labels

In [ ]:
# Plot percentage of JobSatisfaction per Gender
grouped = df.groupby(['Gender'], sort=False)
JobSatisfaction_counts = grouped['DiversityImportant'].value_counts(normalize=True, sort=False)

Satisfaction_data = [
    {'DiversityImportant': JobSatisfaction, 'Gender': Gender, 'percentage': percentage*100} for 
    (Gender, JobSatisfaction), percentage in dict(JobSatisfaction_counts).items()
]

df_Satisfaction = pd.DataFrame(Satisfaction_data)

p = sns.barplot(x="DiversityImportant", y="percentage", hue="Gender", data=df_Satisfaction)
_ = plt.setp(p.get_xticklabels(), rotation=90)  # Rotate labels

## Challenging per years

In [ ]:
#For how many years did you code as part of your job?
data["YearsCodedJobPast"].unique()
data["YearsCodedJobPast"].value_counts()[0:10].plot(kind='bar',figsize=(12,10));

In [ ]:
# data["ChallengeMyself"].unique()
# data[["YearsCodedJobPast","ChallengeMyself"]].value_counts()[0:10].plot(kind='bar', stacked=True,figsize=(12,10));

#For how many years did you code as part of your job?
ax = sns.countplot(x="YearsCodedJobPast", hue="ChallengeMyself", data=df)
plt.xticks(rotation=90)

In [ ]:
# data["HoursPerWeek"]

# # data["ChallengeMyself"].unique()
# # data[["YearsCodedJobPast","ChallengeMyself"]].value_counts()[0:10].plot(kind='bar', stacked=True,figsize=(12,10));

# import seaborn as sns
# #For how many years did you code as part of your job?
# ax = sns.countplot(x="YearsCodedJobPast", hue="ChallengeMyself", data=df)
# plt.xticks(rotation=90)
df2= data[["HoursPerWeek","YearsCodedJobPast"]]

# df2.plot(kind='box')

sns.boxplot(x="YearsCodedJobPast", y="HoursPerWeek", data=df2)
plt.xticks(rotation=90)

# Last Question

In [ ]:
benefits = data["ImportantBenefits"].str.split(pat=";",expand=True)
t=[]
t.append(benefits[0].unique().tolist())
t.append(benefits[1].unique().tolist())
t.append(benefits[2].unique().tolist())
t.append(benefits[3].unique().tolist())
t.append(benefits[4].unique().tolist())

flat_list = [item for sublist in t for item in sublist]

set(flat_list)

In [ ]:
df = df.drop('Respondent', 1)

# Correlation matrix
corr = df.corr()

# Create a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up figure
f, ax = plt.subplots(figsize=(15, 12))

# Plot the heatmap
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});

In [ ]:
plt.figure(figsize=(12,5));
sns.jointplot(x= data['JobSatisfaction'],y=data['Salary'],kind='kde');

In [ ]:
# Calculate average job satisfaction and salary for major countries

df = df.dropna(subset=['JobSatisfaction','Salary'], axis=0)
major_countries = df['Country'].value_counts()[:10].keys()

sal_mean = []
sat_mean = []

for i in range(len(major_countries)):
    sat_mean.append(df.loc[df['Country'] == major_countries[i]]['JobSatisfaction'].mean())
    sal_mean.append(df.loc[df['Country'] == major_countries[i]]['Salary'].mean())
    
# Compare these values by a scatter plot

plt.title("Job Satisfaction & Salary top 10 Countries")
plt.ylabel("Average Job Satisfaction")
plt.xlabel("Average Salary")
plt.scatter(sal_mean, sat_mean)
plt.show()